###Monter Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

Mounted at /content/drive


### Installation

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.5 MB/s eta 0:00:00


### Necessary Imports

In [ ]:
import math
from scipy import stats
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torch import nn, Tensor
from torch.autograd import Variable
import torch
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification

### Configuration des chemins

In [ ]:
train_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_train.txt'#--train_path'
val_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_val.txt'#--val_path'
test_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_test.txt'#--test_path'

model_saving_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/outputs'#--model_saving_path' folder
test_saving_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/ooo.txt'#--test_saving_path'


### Configuration du modèl

In [ ]:
torch.manual_seed(0)

pre_trained_model_name_or_path='roberta-base'#or path savedmodel

# Create Model with specified optimizer and loss function
##############################################################

model = RobertaForSequenceClassification.from_pretrained(pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)


model_name='roberta-base' #'--model_name' or roberta-ft

if model_name == 'roberta-base':
			max_epochs = 30
			lr = 0.00001
			batch_size = 128
			cuda = True
			max_len = 50
elif model_name == 'roberta-ft':
			max_epochs = 30
			lr = 0.00001
			batch_size = 128
			cuda = True
			max_len = 50
else:
			max_epochs = 30
			lr = 0.0001
			batch_size = 128
			cuda = True
			max_len = 50
          
if cuda:
    model.cuda()

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

### Functions

In [ ]:
def padding(text, pad, max_len = 50):

    return text if len(text) >= max_len else (text + [pad] * (max_len-len(text)))

def encode_batch(text, berts, max_len = 50):
    tokenizer = berts[0]
    t1 = []
    for line in text:
        t1.append(padding(tokenizer.encode(line,add_special_tokens = True, max_length = max_len,truncation=True),tokenizer.pad_token_id,max_len))
    return t1


def data_iterator(train_x, train_y, batch_size = 64):
    n_batches = math.ceil(len(train_x) / batch_size)
    for idx in range(n_batches):
        x = train_x[idx *batch_size:(idx+1) * batch_size]
        y = train_y[idx *batch_size:(idx+1) * batch_size]
        yield x, y
        

def get_metrics(model, test_x, test_y, tokenizer, test = False, save_path='test_prediction_final.txt'):
    all_preds = []
    test_iterator = data_iterator(test_x, test_y, batch_size=64)
    all_y = []
    all_x = []
    model.eval()
    for x, y in test_iterator:
        ids = encode_batch(x, (tokenizer,model), max_len = max_len)
        with torch.no_grad():
            if cuda:
                input_ids = Tensor(ids).cuda().long()                
                labels = torch.cuda.FloatTensor(y)
            else:
                input_ids = Tensor(ids).long()
                labels = torch.FloatTensor(y)
            outputs = model(input_ids, labels=labels)
            loss, y_pred = outputs[:2]

        predicted = y_pred.cpu().data
        all_preds.extend(predicted.numpy())
        all_y.extend(y)
        all_x.extend(x)

    all_res = np.array(all_preds).flatten()
    if test and save_path:
        with open(save_path, 'w') as w:
            for i in range(len(all_y)):
                if i < 2:
                    print(all_x[i], all_res[i], test_y[i])
                w.writelines(all_x[i] + '\t' + str(all_y[i]) + '\t' + str(all_res[i]) + '\n')

    #print('pearson r:', stats.pearsonr(all_res, all_y)[0])
    score = 0
    return loss,stats.pearsonr(all_res, all_y)[0]

def run_epoch(model, train_data, val_data, tokenizer, optimizer):
    train_x, train_y = train_data[0], train_data[1]
    val_x, val_y = val_data[0], val_data[1]
    iterator = data_iterator(train_x, train_y, batch_size)
    train_losses = []
    val_accuracies = []
    losses = []

    for i, (x,y) in tqdm(enumerate(iterator),total=int(len(train_x)/batch_size)):
        #print('iteration', i)
        model.zero_grad()

        ids = encode_batch(x, (tokenizer,model), max_len = max_len)


        if cuda:
            input_ids = Tensor(ids).cuda().long()
            labels = torch.cuda.FloatTensor(y)
        else:
            input_ids = Tensor(ids).long()
            labels = torch.FloatTensor(y)

        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        loss.backward()
        #print('train_loss',loss)
        losses.append(loss.data.cpu().numpy())
        optimizer.step()

        if (i + 1) % 1 == 0:
            #print("Iter: {}".format(i))
            avg_train_loss = np.mean(losses)
            train_losses.append(avg_train_loss)
            #print("\tAverage training loss: {:.5f}".format(avg_train_loss))
            losses = []

            # Evalute Accuracy on validation set
            model.eval()
            all_preds = []
            val_iterator = data_iterator(val_x, val_y, batch_size)
            for x, y in val_iterator:
                ids = encode_batch(x, (tokenizer,model), max_len = max_len)
                #x = Variable(Tensor(x))

                with torch.no_grad():

                    if cuda:
                        input_ids = Tensor(ids).cuda().long()
                        labels = torch.cuda.FloatTensor(y)
                    else:
                        input_ids = Tensor(ids).long()
                        labels = torch.FloatTensor(y)
                    outputs = model(input_ids, labels=labels)
                    loss, y_pred = outputs[:2]

                predicted = y_pred.cpu().data

                all_preds.extend(predicted.numpy())

            
            all_res = np.array(all_preds).flatten()            
            score = (np.square(val_y - all_res)).mean()
            val_accuracies.append(score)
            model.train()

    return train_losses, val_accuracies

def get_test_result(model, test_x, test_y, tokenizer, save_path, ext_test = False, pure_inference=False):
    all_raw = []
    all_preds = []
    all_y = []
    all_x = []
    test_iterator = data_iterator(test_x, test_y, batch_size=256)
    model.eval()
    i = 0
    for x, y in test_iterator:
        print(str(i * 256) + '/' + str(len(test_x)))
        i += 1
        #print(x[:5])
        ids = encode_batch(x, (tokenizer, model), max_len = max_len)
        # x = Variable(Tensor(x))

        with torch.no_grad():
            if cuda:
                input_ids = Tensor(ids).cuda().long()
                #labels = torch.cuda.FloatTensor(y)
            else:
                input_ids = Tensor(ids).long()
                #labels = torch.FloatTensor(y)
            outputs = model(input_ids)
            y_pred = outputs[0]

        predicted = y_pred.cpu().data
        #predicted = torch.max(y_pred.cpu().data, 1)[1]
        all_preds.extend(predicted.numpy())
        #all_raw.extend(y_pred.cpu().data.numpy())
        all_y.extend(y)
        all_x.extend(x)


    #all_res = [1 if i[0] > 0 else -1 for i in all_preds]
    all_res = np.array(all_preds).flatten()
    #all_raw = np.array(all_raw)

    if save_path:
        with open(save_path, 'w') as w:
            if pure_inference:
                 for i in range(len(all_y)):                
                    print(all_x[i], all_res[i])                    
                    w.writelines(all_x[i] + '\t' + str(all_res[i]) + '\n')
            else:
                for i in range(len(all_y)):                
                    print(all_x[i], all_res[i], test_y[i])                    
                    w.writelines(all_x[i] + '\t' + str(all_y[i]) + '\t' + str(all_res[i]) + '\n')
    
    if not pure_inference:
        print('mse:', (np.square(all_y - all_res)).mean())
        print('pearson r:', stats.pearsonr(all_res, all_y)[0])

    if ext_test:
        print('book pearson r:', stats.pearsonr(all_res[:50], all_y[:50])[0])
        print('twitter pearson r:', stats.pearsonr(all_res[50:100], all_y[50:100])[0])
        print('movie pearson r:', stats.pearsonr(all_res[100:150], all_y[100:150])[0])

    return all_res, all_y


### Main

In [ ]:

def get_data(path):
    print('open:',path)
    text = []
    y = []
    with open(path, 'r') as r:
        lines = r.readlines()
        for line in lines:
            line = line.strip('\n').split('\t')
            text.append(line[0])
            if len(line) == 2:
                y.append(float(line[1]))
            elif len(line) == 1:
                y.append(0.0)
            else:
                print('error in loading:',path)
    return text,y




mode = 'train'# or 'internal-test' or 'inference'


tokenizer = RobertaTokenizer.from_pretrained(pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)



### Mode training


In [ ]:
train_text,  train_labels = get_data(train_path)
val_text,  val_labels = get_data(val_path)
test_text,  test_labels = get_data(test_path)

train_x = train_text
train_y = np.array(train_labels)
val_x = val_text
val_y = np.array(val_labels)
model.train()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
##############################################################

train_data = [train_x, train_y]
val_data = [val_x, val_y]

# Get Accuracy of final model
test_x = test_text
test_y = np.array(test_labels)
best_val = 100.0
best_test = 100.0
best_r = 100

for i in range(max_epochs):
    print ("Epoch: {}".format(i))

    train_losses,val_accuracies = run_epoch(model, train_data, val_data, tokenizer, optimizer)
    test_acc,test_r = get_metrics(model, test_x, test_y,  tokenizer, test = True, save_path=test_saving_path)
    #print('Final Test Accuracy: {:.4f}'.format(test_acc))

    print("\tAverage training loss: {:.5f}".format(np.mean(train_losses)))
    print("\tAverage Val MSE: {:.4f}".format(np.mean(val_accuracies)))
    if np.mean(val_accuracies) < best_val:
        best_val = np.mean(val_accuracies)
        best_test = test_acc
        best_r = test_r
        if i >= 1 and model_saving_path:
            model.save_pretrained(model_saving_path)
            tokenizer.save_pretrained(model_saving_path)


print('model saved at', model_saving_path)
print('best_val_loss:', best_val)
print('best_test_loss:',best_test)
print('best_test_pearsonr:',best_r)

#test_acc = get_metrics(model, test_x, test_y, config, tokenizer, test = True)
#if args.model_saving_path:
#    model.save_pretrained(args.model_saving_path)
#    tokenizer.save_pretrained(args.model_saving_path)



open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_train.txt
open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_val.txt
open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_test.txt
Epoch: 0


15it [00:22,  1.50s/it]


Besides medical equipment or technology, what is a common, everyday item that would confuse someone the most from the middle ages? -0.034892585 -0.1302005785547401
Can I eat and pee at the same time? -0.047029573 0.08370097250899593
	Average training loss: 0.10142
	Average Val MSE: 0.0887
Epoch: 1


15it [00:23,  1.54s/it]


Besides medical equipment or technology, what is a common, everyday item that would confuse someone the most from the middle ages? -0.08259615 -0.1302005785547401
Can I eat and pee at the same time? -0.07534729 0.08370097250899593
	Average training loss: 0.07601
	Average Val MSE: 0.0659
Epoch: 2


...

Besides medical equipment or technology, what is a common, everyday item that would confuse someone the most from the middle ages? -0.23732258 -0.1302005785547401
Can I eat and pee at the same time? -0.032278046 0.08370097250899593
	Average training loss: 0.01179
	Average Val MSE: 0.0249
Epoch: 28


15it [00:22,  1.50s/it]


Besides medical equipment or technology, what is a common, everyday item that would confuse someone the most from the middle ages? -0.14268327 -0.1302005785547401
Can I eat and pee at the same time? 0.034724157 0.08370097250899593
	Average training loss: 0.01072
	Average Val MSE: 0.0259
Epoch: 29


15it [00:22,  1.51s/it]


Besides medical equipment or technology, what is a common, everyday item that would confuse someone the most from the middle ages? -0.20946378 -0.1302005785547401
Can I eat and pee at the same time? -0.032980762 0.08370097250899593
	Average training loss: 0.01151
	Average Val MSE: 0.0266
model saved at /content/drive/MyDrive/Colab Notebooks/QIL/data/outputs
best_val_loss: 0.024680131073151135
best_test_loss: tensor(0.0192, device='cuda:0')
best_test_pearsonr: 0.8117268784473094


### Mode Internal Test
The best model will be saved at `outputs/`

after training, to get the score on our annotated test and out-domain set,``` \
--pre_trained_model_name_or_path=outputs \
--predict_data_path=data/annotated_question_intimacy_data/final_external.txt 

In [ ]:
# pre_trained_model_name_or_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/outputs'
pre_trained_model_name_or_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/out2/best-model'

predict_data_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_external.txt'#--predict_data_path'

model = RobertaForSequenceClassification.from_pretrained(pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)
# if cuda:
model.cuda()

val_text,  val_labels = get_data(val_path)
test_text,  test_labels = get_data(test_path)
final_test_text,final_test_y = get_data(predict_data_path)

print('external_test:')
test_result, test_score = get_test_result(model, final_test_text, final_test_y, tokenizer,save_path=test_saving_path, ext_test = True)

print('val:')
test_result, test_score = get_test_result(model, val_text, val_labels, tokenizer,save_path=None, ext_test = False)

print('test:')
test_result, test_score = get_test_result(model, test_text, test_labels, tokenizer,save_path=None, ext_test = False)


Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/QIL/data/out2/best-model were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /co

open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_val.txt
open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_test.txt
open: /content/drive/MyDrive/Colab Notebooks/QIL/data/annotated_question_intimacy_data/final_external.txt
external_test:
0/150
Oh, Mr. Stephens, is there nothing we could do? -0.1535802 -0.05413701329856845
what is an intimate friend? -0.15055382 0.20087224199975143
How can you endure this kind of life? -0.15656313 0.43400000918507603
They are frightened--the little ones? -0.15018514 -0.06120741806706436
No! Go now, and take me with you! How should I find my way to London alone, or seek out this man Bradbury, or your mother? -0.14958134 -0.2766953082825539
What are you doing that for? -0.15145521 0.015148892275376288
Know you not that this is a manse? -0.14613369 -0.4536741581121467
But what's the sense in playing him , Chip? -0.1554547 0.15421128819114172
Now then, sleepy, what's yer g

### Mode inference 

to run the fine-tuned model over your own data, prepare a file with a list of input text like `data/inference.txt` ```\
--predict_data_path=data/inference.txt \
--test_saving_path=ooo.txt

In [ ]:
predict_data_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/inference.txt'#--predict_data_path'
test_saving_path='/content/drive/MyDrive/Colab Notebooks/QIL/data/ooo.txt'#--test_saving_path'

final_test_text,final_test_y = get_data(predict_data_path)
test_result, test_score = get_test_result(model, final_test_text, final_test_y, tokenizer,save_path=test_saving_path, ext_test = False, pure_inference=True)


open: /content/drive/MyDrive/Colab Notebooks/QIL/data/inference.txt
0/3
Do you love me? -0.14744307
What's your favorite hobby? -0.14598438
are you student? -0.14496452
